In [6]:
import numpy as np
import os

In [9]:
def overlapping_read_write(file_gms, seq_file, out_file, name, overlap):
    #work with files
    
    file = open(file_gms, 'r')
    file_reader = np.loadtxt(file, dtype ='str', skiprows=11, delimiter='\t')
    file.close()
    genes_positions = []
    for gene in file_reader:
        genes_positions.append(gene.split(' '))
    for gene in genes_positions:
        if ('' in gene):
            while('' in gene):
                gene.remove('')
    file_seq = open(seq_file, 'r')
    file_reader = np.loadtxt(file_seq, dtype ='str', skiprows=1, delimiter='\n')
    file_seq.close()
    
    #Searching regions
    
    upstream_regions = []
    intersection = []
    sequence = str(file_reader)
    for gene in genes_positions:
        s = []
        strn = ''
        s.append(str(gene[0]))
        if ('<' not in gene[2]) and ('>' not in gene[3]) and (int(gene[2]) > 50) and (int(gene[3]) + 49 < len(sequence) and (int(gene[2]) + int(gene[4]) < len(sequence))):
        
            if gene[1] == '+':
                
                for i in range(int(gene[2]) - 49, int(gene[2]) + 1):
                    strn += str(sequence[i])
                s.append(strn)
                s.append(int(gene[2]) - 50)
                s.append(int(gene[2]))
            
            if gene[1] == '-':
                
                for i in range(int(gene[3]) , int(gene[3]) + 50):
                    strn += str(sequence[i])
                s.append(strn[::-1])
                s.append(int(gene[3]))
                s.append(int(gene[3]) + 50)
             
            if (int(gene[2]) < 50 and gene[1] == '+'):
                
                for i in range(0, int(gene[2]) + 1):
                    strn += str(sequence[i])
                s.append(strn)
                s.append(0)
                s.append(int(gene[2]))
                
            if (int(gene[3]) + 49 > len(sequence) and gene[2] == '-'):
                
                for i in range(int(gene[3]), len(sequence) - 1):
                    strn += str(sequence[i])
                s.append(strn[::-1])
                s.append(int(gene[3]))
                s.append(len(sequence))
        
            upstream_regions.append(s)

  #  upstream_regions_complement = []
  #  for i in range(len(upstream_regions)):
  #      upstream_region = ''
  #      for k in upstream_regions[i][1]:
  #          for j in k:
  #              if (j == 'T'):
  #                  upstream_region += 'A'
  #              if (j == 'A'):
  #                  upstream_region += 'T'
  #              if (j == 'C'):
  #                  upstream_region += 'G'
  #              if (j == 'G'):
  #                  upstream_region += 'C'
  #      upstream_regions_complement.append(upstream_region)
        
        
    #Find overlapping regions
    overlapping = []
    for i in upstream_regions:
        for j in genes_positions:
            if ('<' not in j[2]) and ('>' not in j[3]):
                if (i[0] != j[0] and len(i) >= 3 and len(j) > 3):
                    if (int(j[3]) in range(int(i[2]), int(i[3] + 1))) or (int(j[2]) in range(int(i[2]), int(i[3] + 1))): 
                        s = [i[0], j[0]]
                        k = [j[0], i[0]]
                        if (s not in overlapping and k not in overlapping):
                            get = str(s[0]) + ' ' + str(s[1])
                            overlapping.append(get)
                            
                            
    #output_writing
    out = open(out_file, 'w')
    for i in range(len(upstream_regions)):
        out.write('>' + name + '--GMS2_')
        out.write(str(upstream_regions[i][0]) + '--UPS')
        out.write('\n')
        out.write(str(upstream_regions[i][1]))
        out.write('\n')
  #      out.write(str(upstream_regions_complement[i]))
  #       out.write('\n')
   # for i in genes_positions:
   #     if ('<' in i[2]) or ('>' in i[3]) : 
   #         out.write('>' + name + '--GMS2_')
   #         out.write(i[0])
   #         out.write('\nNo upsream region found\n')
    out.close()
    
    out = open(overlap, 'w')
    #out.write('Overlapping_regions')
    #out.write('\n')

    for i in range(len(overlapping)):
        out.write(str(overlapping[i]))
        out.write('\n')
    out.close()
    return 0

In [10]:
for root, dirs, files in os.walk('/home/emil/Desktop/Science/data/gms2_viruses_output'):
    for file in files:
        path = os.path.join(root, file)
        s = path.split('/')
        name = s[len(s)-1].replace('_gms2.out','')
        overlapping = '/home/emil/Desktop/Science/data/overlapping_upstream_regions_and_genes/' + name + '_overlapping.txt' 
        sequence = '/home/emil/Desktop/Science/data/viruses_genomes/' + name
        outname = '/home/emil/Desktop/Science/data/upstream_regions/' + name +'_UPS.fasta'
        overlapping_read_write(path, sequence, outname, name, overlapping)